# Structural variant-eQTL
- Refer to Scott et al. 2021 and Chiang et al. 2017, where SVs were called in GTEx WGS, merged with SNVs and indels, and FastQTL was used to map association with gene expression
- Primary analysis here: running linear regression of gene expression with SVs only

In [1]:
library(data.table)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()



In [2]:
setwd("~/project-gandalm/isoform_twas/eqtl_new/structural/")

In [3]:
load("output_18invs.RData")

In [4]:
ls()

[1] "inv_info" "recorder"

In [5]:
inv_info

,seqnames,start,end,width,strand,scoreInvHap.name,Cytogenetic.location,Inv.freq,Haplotypes,Num.SNPs
,<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
inv1_004,chr1,92131841,92132615,775,*,inv1_004,1p22.1,11.23,2,6.000
inv1_008,chr1,197756784,197757982,1199,*,inv1_008,1q31.3,19.68,2,5.000
inv2_002,chr2,33764554,33765272,719,*,inv2_002,2p22.3,15.11,2,6.000
inv2_013,chr2,139004949,139009203,4255,*,inv2_013,2q22.1,71.47,2,13.000
inv3_003,chr3,162545362,162547641,2280,*,inv3_003,3q26.1,56.16,4,6.000
inv6_002,chr6,31009222,31010095,874,*,inv6_002,6p21.33,63.12,2,5.000
inv6_006,chr6,130848198,130852318,4121,*,inv6_006,6q23.1,6.56,2,12.000
inv7_003,chr7,31586765,31592019,5255,*,inv7_003,7p14.3,23.56,2,11.000
inv7_005,chr7,54302450,54376389,73940,*,inv7_005,7p11.2,50.39,4,180.000


In [7]:
head(recorder)

,inv1_004,inv1_008,inv10_005,inv11_001,inv11_004,inv12_004,inv16_009,inv17_007,inv2_002,inv2_013,inv21_005,inv6_002,inv6_006,inv7_003,inv7_005,inv7_011,inv7_014,inv8_001
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1038,NN,NN,NN,NI,II,NN,NI,II,NN,II,NaNb,II,NN,NI,NaNa,NN,NN,NN
1039,NI,NN,NI,NI,NaI,NN,NN,NN,NN,II,NaI,NN,NN,NI,NaIb,II,NN,NI
1046,NN,NN,NN,NN,NaNa,NN,NN,NN,NN,NN,NaI,NN,NN,NI,NaIa,NI,NN,II
1102,NN,NN,NN,NN,NbI,NN,NN,NI,NN,NI,NcI,II,NN,NN,IaIb,II,NI,NI
1107,NN,NN,NN,NI,NaI,NN,II,NN,NN,NN,II,II,II,NI,IaIb,NI,NI,NI
1110,NN,NN,NN,NN,NaNa,NN,NN,NN,NN,II,NcI,NN,NN,NI,IbIb,NI,NN,NI


In [10]:
expr <- fread("../data/gene.counts.scaled.normalized.bed.gz", data.table = F)
head(expr)

,#Chr,start,end,ID,1474,1496,1500,1502,1507,1511,...,1558,1607,1707.1,1835,1851,1890,1923,2193,849,898
,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,11868,11869,ENSG00000223972,3.135499,3.860746,2.991053,3.402323,1.683311,2.979528,...,3.545430,2.794408,3.219573,3.270080,3.033587,3.719546,3.528669,1.795879,1.795879,1.795879
2,1,29553,29554,ENSG00000243485,2.019259,2.019259,3.337607,3.284138,2.019259,4.117352,...,1.739895,1.739895,3.364551,2.579396,3.357388,2.792396,2.825834,3.070580,3.492360,2.796174
3,1,29569,29570,ENSG00000227232,7.976420,7.983908,8.418516,8.327781,7.848723,8.723322,...,6.898168,8.165533,9.093119,7.508564,8.412931,8.818620,7.683092,8.549899,7.261408,7.765424
4,1,36080,36081,ENSG00000237613,3.965445,4.274929,4.684834,4.872262,4.790440,3.521006,...,2.993636,4.926470,2.993636,2.993636,2.993636,2.993636,2.993636,5.123510,2.993636,2.993636
5,1,91104,91105,ENSG00000239945,1.850485,3.306370,2.977209,3.985883,1.850485,1.850485,...,2.157703,2.157703,2.157703,3.728385,5.165983,3.846017,2.157703,2.157703,2.157703,4.511053
6,1,131024,131025,ENSG00000233750,4.110258,4.971833,4.274072,4.756833,5.022698,3.450271,...,4.431115,4.068149,4.380002,3.577251,3.459137,4.259683,3.918486,3.981142,4.821110,5.015399


In [12]:
cov <- fread("../data/90hcp_cov.txt", data.table = F, header = T)
head(cov)

,id,1474,1496,1500,1502,1507,1511,1513,1525,1531,...,1558,1607,1707.1,1835,1851,1890,1923,2193,849,898
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,PC1,0.00898199,-0.0076338,0.0111147,0.00487142,0.00960893,-0.01488,0.00883737,-0.0266981,-0.0253829,...,0.00967447,-0.0252683,0.00965381,0.00960237,0.00979039,0.00950035,0.00960529,0.00989866,0.00702467,0.00983834
2,PC2,-0.00035727,-0.000843408,-0.0158545,0.000257063,-0.00326613,-0.00113582,-0.00241496,-0.0022033,-0.00247448,...,-0.0208489,-0.00395539,-0.0206395,-0.0210822,-0.0204132,-0.0205727,-0.0206059,-0.0210826,0.0245783,-0.0205709
3,PC3,-0.0248268,-0.0141886,-0.0101601,-0.0222758,-0.020871,-0.00907471,-0.021557,-0.00118133,-0.00152521,...,-0.00482939,-0.00111922,-0.00453659,-0.00526304,-0.00508921,-0.00534413,-0.00518309,-0.00544843,-0.00538244,-0.00537699
4,PC4,-0.0321383,-0.0175555,-0.000733046,-0.0315899,-0.0245335,-0.0106596,-0.0266302,0.000414128,0.000549499,...,0.0229196,0.0138587,0.0222114,0.0226635,0.0214958,0.0226123,0.0221212,0.0239712,0.0252937,0.0216689
5,PC5,0.000571238,-0.00167298,-0.0126722,0.00128243,-0.00225702,-0.00145859,-0.00174134,-0.00474446,-0.00445944,...,0.0685564,0.0750816,0.0672512,0.0693604,0.0685406,0.06972,0.0690513,0.0679772,0.0693614,0.0681412
6,sex,M,F,M,M,M,M,M,M,F,...,F,F,F,M,F,F,F,M,F,F


In [13]:
rel <- fread("../../genotype/all_data/isec_R2_greater_than_3/ancestry/related.txt", data.table = F)$V1
rel

[1] "1614"   "11602"  "Br1779" "Br2394" "Br2402" "Br2411" "Br2412" "1713"  
 [9] "11841"  "Br1794" "HSB148" "HSB107" "HSB154" "HSB100"

**Remove related subjects**

In [21]:
cov <- cov[,!names(cov) %in% rel]
dim(cov)

[1]  97 630

In [24]:
recorder <- recorder[!rownames(recorder) %in% rel,]
dim(recorder)

[1] 629  18

In [25]:
expr <- expr[,!names(expr) %in% rel]
dim(expr)

[1] 31947   633

**Linear regression `resid(expr) ~ SV`**
- QTLtools first regresses out the covariates from the phenotype data, followed by running the linear regression between the phenotype residuals and the genotype
- If the `--normal` and `--cov` are provided at the same time, then the residuals after the covariate correction are rank normal transformed
- Here also testing the coded regression with `lm`

**1. First regress out covariates from expression**

In [26]:
rownames(cov) <- cov$id
cov <- cov[,-1]
cov[cov=="F"] <- 0
cov[cov=="M"] <- 1
cov <- t(cov)
cov <- cbind(1, cov)
storage.mode(cov) <- "numeric"

In [27]:
head(cov)

,,PC1,PC2,PC3,PC4,PC5,sex,HCP1,HCP2,HCP3,...,HCP82,HCP83,HCP84,HCP85,HCP86,HCP87,HCP88,HCP89,HCP90,age
1474,1,0.00898199,-0.000357270,-0.02482680,-0.032138300,0.000571238,1,0.02489871,-0.4019976,-0.3388445,...,0.80657916,-0.37321124,0.58568964,-0.061274183,-0.23737501,-0.41832559,-0.41279687,-1.00632175,0.29851416,-0.4986301
1496,1,-0.00763380,-0.000843408,-0.01418860,-0.017555500,-0.001672980,0,-0.20898506,0.1315736,0.4785638,...,0.10338800,0.56528105,-0.07663771,-0.004087691,0.21658413,-0.18350357,-0.05032608,-0.05653829,0.03211184,-0.4602740
1500,1,0.01111470,-0.015854500,-0.01016010,-0.000733046,-0.012672200,1,-0.03128920,-0.4912983,-0.1810369,...,-0.18134092,-0.09036451,0.11534568,0.144869882,0.11412663,-0.01423129,0.25413426,-0.48506890,-0.09221297,-0.4794521
1502,1,0.00487142,0.000257063,-0.02227580,-0.031589900,0.001282430,1,-0.03733652,0.2831048,0.1061886,...,0.07935134,0.11610668,0.06276578,0.401644541,-0.02963446,0.12413614,-0.13586945,-0.08789221,0.00116774,-0.4410959
1507,1,0.00960893,-0.003266130,-0.02087100,-0.024533500,-0.002257020,1,0.15308896,-0.3783080,0.5525917,...,-0.10455167,0.31311166,-0.23432174,0.287902870,0.18724611,0.19281353,0.16318990,0.04997490,0.05738023,-0.4602740
1511,1,-0.01488000,-0.001135820,-0.00907471,-0.010659600,-0.001458590,1,-0.11345797,-0.3760258,-0.8350841,...,0.36750293,-0.14613559,0.44909112,0.166787756,0.12988756,-0.48845279,-0.77368296,-0.57988925,0.42413132,-0.4986301


In [30]:
datExpr <- expr[,4:ncol(expr)]
rownames(datExpr) <- datExpr$ID
datExpr <- datExpr[,-1]

In [31]:
head(datExpr)

,1474,1496,1500,1502,1507,1511,1513,1525,1531,1533,...,1558,1607,1707.1,1835,1851,1890,1923,2193,849,898
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000223972,3.135499,3.860746,2.991053,3.402323,1.683311,2.979528,1.683311,1.683311,3.583774,1.683311,...,3.545430,2.794408,3.219573,3.270080,3.033587,3.719546,3.528669,1.795879,1.795879,1.795879
ENSG00000243485,2.019259,2.019259,3.337607,3.284138,2.019259,4.117352,2.019259,2.019259,2.019259,2.019259,...,1.739895,1.739895,3.364551,2.579396,3.357388,2.792396,2.825834,3.070580,3.492360,2.796174
ENSG00000227232,7.976420,7.983908,8.418516,8.327781,7.848723,8.723322,8.047395,8.643141,8.822918,7.658501,...,6.898168,8.165533,9.093119,7.508564,8.412931,8.818620,7.683092,8.549899,7.261408,7.765424
ENSG00000237613,3.965445,4.274929,4.684834,4.872262,4.790440,3.521006,4.333095,4.779410,5.227346,-0.483929,...,2.993636,4.926470,2.993636,2.993636,2.993636,2.993636,2.993636,5.123510,2.993636,2.993636
ENSG00000239945,1.850485,3.306370,2.977209,3.985883,1.850485,1.850485,1.850485,1.850485,1.850485,4.087958,...,2.157703,2.157703,2.157703,3.728385,5.165983,3.846017,2.157703,2.157703,2.157703,4.511053
ENSG00000233750,4.110258,4.971833,4.274072,4.756833,5.022698,3.450271,4.377471,3.433721,4.665800,4.482790,...,4.431115,4.068149,4.380002,3.577251,3.459137,4.259683,3.918486,3.981142,4.821110,5.015399


In [32]:
Y <- as.matrix(datExpr)
X <- as.matrix(cov)

beta <- (solve(t(X) %*% X) %*% t(X)) %*% t(Y)
datExpr_regressed <- Y - t(X[,-1] %*% beta[-1,])

In [33]:
head(datExpr_regressed)

,1474,1496,1500,1502,1507,1511,1513,1525,1531,1533,...,1558,1607,1707.1,1835,1851,1890,1923,2193,849,898
ENSG00000223972,2.867789,3.296470,2.200791,2.684605,1.4565833,3.400122,1.017490,0.9603317,2.611516,1.5514351,...,2.852729,2.339485,2.943993,3.169074,2.995443,3.501674,2.804495,1.687725,1.808905,1.666353
ENSG00000243485,2.060708,1.581782,2.679413,2.830359,1.4140588,3.376425,1.656341,2.0834450,1.724222,1.4361136,...,1.237803,1.312254,2.414263,1.898056,2.520261,2.140663,2.337385,2.204175,2.321048,2.050840
ENSG00000227232,8.811202,8.280995,8.931025,8.949145,8.3683365,9.113657,8.919081,8.5016701,8.582253,8.5895124,...,7.633874,8.261629,8.822906,7.728855,8.552273,8.871003,8.440336,9.129610,7.818537,8.283419
ENSG00000237613,4.895634,4.615218,4.995389,4.928265,4.8845307,4.361184,4.768870,5.3094919,4.451794,0.2609459,...,3.829474,4.982749,4.022271,4.088187,4.231415,3.810719,3.647314,5.210864,4.068420,4.540590
ENSG00000239945,1.322099,2.477211,2.435626,2.840091,0.6167818,1.252557,1.184240,1.4249191,2.130361,2.6729505,...,1.522859,2.418559,1.408638,2.505883,4.130191,3.238845,1.706037,1.256406,1.003519,2.954425
ENSG00000233750,4.261517,4.511196,4.226093,4.049178,4.7536951,4.284809,4.366938,3.0368784,4.459563,4.1288294,...,4.438547,4.543358,4.305752,4.365838,4.230395,3.963106,4.015720,4.033887,4.405711,5.244836


In [48]:
beta_round <- round(beta, 2)
head(beta_round)

,ENSG00000223972,ENSG00000243485,ENSG00000227232,ENSG00000237613,ENSG00000239945,ENSG00000233750,ENSG00000238009,ENSG00000268903,ENSG00000237683,ENSG00000239906,...,ENSG00000148399,ENSG00000165724,ENSG00000197070,ENSG00000203993,ENSG00000181090,ENSG00000148408,ENSG00000203987,ENSG00000276388,ENSG00000159247,ENSG00000233013
,2.58,2.05,8.58,4.46,2.14,4.30,6.41,2.18,6.35,3.18,...,9.67,9.08,8.34,8.60,11.87,11.42,3.97,2.27,5.06,2.98
PC1,-5.58,0.54,0.10,2.45,-12.24,-1.34,0.53,-0.09,-0.87,-5.62,...,-0.31,-0.33,0.15,0.93,4.43,-1.22,0.57,4.10,-5.26,-7.55
PC2,-1.80,8.81,-1.54,-3.55,11.14,0.42,0.83,7.77,2.12,-4.83,...,-0.59,0.70,0.24,0.91,1.25,0.79,0.61,0.05,6.76,-1.61
PC3,-9.17,-12.25,2.61,4.72,3.04,-1.40,6.23,-0.18,-0.95,-0.25,...,0.37,0.64,-0.32,0.20,-1.31,-1.30,-0.33,-3.09,-5.33,14.45
PC4,0.63,10.36,-0.32,-0.44,1.66,0.15,-1.29,-3.44,4.68,-1.63,...,0.12,0.08,-1.19,0.31,0.96,0.61,0.53,1.02,8.32,-8.17
PC5,-1.12,-3.07,0.43,1.68,-2.10,-0.45,0.18,-0.81,-1.63,0.11,...,0.27,-0.37,0.00,-0.16,0.94,-0.24,-0.21,0.02,-1.55,-3.31


In [39]:
dim(beta_round)

[1]    98 31947

**Test with lm function (https://datascienceplus.com/linear-regression-from-scratch-in-r/)**

In [44]:
lm.mod <- lm(t(Y) ~ X)
lm.betas <- round(lm.mod$coefficients, 2)

In [46]:
dim(lm.betas)
head(lm.betas)

[1]    99 31947

,ENSG00000223972,ENSG00000243485,ENSG00000227232,ENSG00000237613,ENSG00000239945,ENSG00000233750,ENSG00000238009,ENSG00000268903,ENSG00000237683,ENSG00000239906,...,ENSG00000148399,ENSG00000165724,ENSG00000197070,ENSG00000203993,ENSG00000181090,ENSG00000148408,ENSG00000203987,ENSG00000276388,ENSG00000159247,ENSG00000233013
(Intercept),2.58,2.05,8.58,4.46,2.14,4.30,6.41,2.18,6.35,3.18,...,9.67,9.08,8.34,8.60,11.87,11.42,3.97,2.27,5.06,2.98
X,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
XPC1,-5.58,0.54,0.10,2.45,-12.24,-1.34,0.53,-0.09,-0.87,-5.62,...,-0.31,-0.33,0.15,0.93,4.43,-1.22,0.57,4.10,-5.26,-7.55
XPC2,-1.80,8.81,-1.54,-3.55,11.14,0.42,0.83,7.77,2.12,-4.83,...,-0.59,0.70,0.24,0.91,1.25,0.79,0.61,0.05,6.76,-1.61
XPC3,-9.17,-12.25,2.61,4.72,3.04,-1.40,6.23,-0.18,-0.95,-0.25,...,0.37,0.64,-0.32,0.20,-1.31,-1.30,-0.33,-3.09,-5.33,14.45
XPC4,0.63,10.36,-0.32,-0.44,1.66,0.15,-1.29,-3.44,4.68,-1.63,...,0.12,0.08,-1.19,0.31,0.96,0.61,0.53,1.02,8.32,-8.17


In [50]:
results <- data.frame(our.results = beta_round[2:98,1:3], lm.results = lm.betas[3:99,1:3])
head(results)

,our.results.ENSG00000223972,our.results.ENSG00000243485,our.results.ENSG00000227232,lm.results.ENSG00000223972,lm.results.ENSG00000243485,lm.results.ENSG00000227232
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PC1,-5.58,0.54,0.10,-5.58,0.54,0.10
PC2,-1.80,8.81,-1.54,-1.80,8.81,-1.54
PC3,-9.17,-12.25,2.61,-9.17,-12.25,2.61
PC4,0.63,10.36,-0.32,0.63,10.36,-0.32
PC5,-1.12,-3.07,0.43,-1.12,-3.07,0.43
sex,-0.01,0.02,-0.07,-0.01,0.02,-0.07


**TODO: Rank-based inverse normalize transformation to N(0, 1) of the phenotype residuals**

**2. TODO linear regression with SVs**

In [51]:
df <- cbind(t(datExpr_regressed), recorder)
head(df)

,ENSG00000223972,ENSG00000243485,ENSG00000227232,ENSG00000237613,ENSG00000239945,ENSG00000233750,ENSG00000238009,ENSG00000268903,ENSG00000237683,ENSG00000239906,...,inv2_002,inv2_013,inv21_005,inv6_002,inv6_006,inv7_003,inv7_005,inv7_011,inv7_014,inv8_001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1474,2.867789,2.060708,8.811202,4.895634,1.3220987,4.261517,7.036881,1.4847741,6.176734,1.903290,...,NN,II,NaNb,II,NN,NI,NaNa,NN,NN,NN
1496,3.296470,1.581782,8.280995,4.615218,2.4772108,4.511196,6.417705,0.6439407,6.539432,3.479306,...,NN,II,NaI,NN,NN,NI,NaIb,II,NN,NI
1500,2.200791,2.679413,8.931025,4.995389,2.4356263,4.226093,7.247070,4.7238855,6.082197,3.783942,...,NN,NN,NaI,NN,NN,NI,NaIa,NI,NN,II
1502,2.684605,2.830359,8.949145,4.928265,2.8400906,4.049178,5.940432,0.9745495,6.633310,3.903433,...,NN,NI,NcI,II,NN,NN,IaIb,II,NI,NI
1507,1.456583,1.414059,8.368337,4.884531,0.6167818,4.753695,4.962535,3.8212697,5.751311,2.286177,...,NN,NN,II,II,II,NI,IaIb,NI,NI,NI
1511,3.400122,3.376425,9.113657,4.361184,1.2525571,4.284809,7.274663,1.8349739,6.708908,3.785365,...,NN,II,NcI,NN,NN,NI,IbIb,NI,NN,NI


In [52]:
table(df$inv2_002)


 II  NI  NN 
276   0 353 

In [53]:
model <- lm(ENSG00000223972 ~ inv2_002, data = df)

In [54]:
model


Call:
lm(formula = ENSG00000223972 ~ inv2_002, data = df)

Coefficients:
(Intercept)   inv2_002NN  
    2.59764     -0.02475  


In [55]:
summary(model)


Call:
lm(formula = ENSG00000223972 ~ inv2_002, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.18695 -0.50677  0.04006  0.51405  1.90275 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.59764    0.04360  59.580   <2e-16 ***
inv2_002NN  -0.02475    0.05820  -0.425    0.671    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7243 on 627 degrees of freedom
Multiple R-squared:  0.0002882,	Adjusted R-squared:  -0.001306 
F-statistic: 0.1808 on 1 and 627 DF,  p-value: 0.6709
